# Kinematics Example for the SCARA Robot

![SCARA robot](https://linux-cdn.softpedia.com/screenshots/SCARA-robot_1.jpg)

SCARA stands for Selective Compliance Assembly Robot Arm. It is a two-link manipulator operating in the $x$-$y$ plane, with some form of linear actuator at the end of the manipulator operating in along the $z$ axis.

**TODO:** Better pictures.

![scara-tikz](figures/scara-3d.svg)

## Forward Kinematics

We define the origin of our coordinate system to be centered at the axis of rotation for the first arm, with the $-z$-axis pointing downwards to retain right-handedness of the coordinate system.

Then the forward kinematics equations are simply

$$x = a_2 \cos(\theta_1 + \theta_2) + a_1 \cos(\theta_1)$$
$$y = a_2 \sin(\theta_1 + \theta_2) + a_1 \sin(\theta_1)$$
$$z = q_3$$

Note that practical considerations may require $z = q_3 + c$ for some constant $c$ due to arm widths or limitations of the linear actuator, but we will assume the simpler form of $z = q_3$ in this example.

## Inverse Kinematics

Due to the similarity of this system to the two-link manipulator, the inverse kinematics have already been derived.

$$D = \frac{x^2 + y^2 - a_1^2 - a_2^2}{2 a_1 a_2}$$

$$\theta_1 = \tan^{-1}\frac{y}{x} - \tan^{-1} \frac{a_2\sin \theta_2}{a_1 + a_2\cos\theta_2}$$
$$\theta_2 = \tan^{-1}\frac{\pm\sqrt{1-D^2}}{D}$$
$$q_3 = z$$

Note the dependence of $\theta_1$ on $\theta_2$.

## Python Example for a Straight Line

Begin by importing `numpy` and `matplotlib` for nice numerical and plotting support.

In [ ]:
%config InlineBackend.figure_format = 'svg'
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn

seaborn.set()

Then define the arm lengths as constants for the problem

In [ ]:
a1 = 10
a2 = 8

Then define functions for the forward and inverse kinematics.

In [ ]:
def forward(theta1, theta2, q3):
    """Forward kinematics for a SCARA robot"""
    x = a2 * np.cos(theta1 + theta2) + a1 * np.cos(theta1)
    y = a2 * np.sin(theta1 + theta2) + a1 * np.sin(theta1)
    z = q3

    return x, y, z

In [ ]:
def inverse(x, y, z):
    """Inverse kinematics for a SCARA robot"""
    D = (x**2 + y**2 - a1**2 - a2**2) / (2 * a1 * a2)
    theta2 = np.arctan2(np.sqrt(1 - D**2), D)
    theta1 = np.arctan2(y, x) - np.arctan2(a2 * np.sin(theta2), a1 + a2 * np.cos(theta2))
    q3 = z

    return theta1, theta2, q3

Now consider the following path through the 3D workspace. We parameterize the path as a function of $t$ for programmatic simplicity.

$$x = x_0 + at$$
$$y = y_0 + bt$$
$$z = z_0 + ct$$

where $(x_0, y_0, z_0)$ is an initial point and $\langle a, b, c \rangle$ is the direction of travel.

For this example, we will use $(0, -5, 0)$ as our initial point, and $(5, 5, -5)$ as our ending point. This gives $\langle 1, 2, -1 \rangle$ as the direction of travel, with $0 \leq t \leq 5$?

In Python, we generate points on this line as follows.

In [ ]:
def path(t):
    """A parameterized path in the workspace"""
    x = 0 + 1 * t
    y = -5 + 2 * t
    z = 0 - 1 * t

    return x, y, z

In [ ]:
t = np.arange(start=0, stop=5, step=0.1)
x, y, z = path(t)

In [ ]:
fig = plt.figure()
# Create an Axes3D object, setting the elevation and azimuth
ax = fig.add_subplot(111, projection='3d', elev=20, azim=0)

ax.plot(x, y, z)
plt.title('Desired workspace path')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
plt.show()

Now run the inverse kinematics on the workspace path to compute and plot the corresponding path in the configuration space.

In [ ]:
theta1, theta2, q3 = inverse(x, y, z)

In [ ]:
fig = plt.figure()
# Create an Axes3D object, setting the elevation and azimuth
ax = fig.add_subplot(111, projection='3d', elev=20, azim=0)

ax.plot(theta1, theta2, q3)
plt.title('Computed configuration space path')
ax.set_xlabel('$\\theta_1$')
ax.set_ylabel('$\\theta_2$')
ax.set_zlabel('$q_3$')
plt.show()

Now run the path through the configuration space back through the forward kinematics and plot as a sanity check.

In [ ]:
cx, cy, cz = forward(theta1, theta2, q3)

In [ ]:
fig = plt.figure()
# Create an Axes3D object, setting the elevation and azimuth
ax = fig.add_subplot(111, projection='3d', elev=20, azim=0)

ax.plot(cx, cy, cz)
plt.title('Computed workspace path')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
plt.show()

This looks like the right path, but programmatically verify:

In [ ]:
print(np.allclose(x, cx))
print(np.allclose(y, cy))
print(np.allclose(z, cz))

## Python Example for a 3D Helix

A circular 3D helix is parameterized as follows

$$x = x_0 + r \cos t$$
$$y = y_0 + r \sin t$$
$$z = z_0 + ct$$

where $(x_0, y_0, z_0)$ is the starting point, $r$ is the radius, and $2 \pi c$ is the constant vertical separation between the spirals.

In this example, we use $x = x_0 + r_1 \cos t$ and $y = y_0 + r_2 \sin t$ so that the path is not circular when viewed from above.

In [ ]:
def path(t):
    """A parameterized path in the workspace"""
    x = 15 * np.cos(t)
    # Use a different radius for the y coordinate to prevent tracing a circle
    y = 4 * np.sin(t)
    z = - 1.0 * t / (10 * np.pi)

    return x, y, z

In [ ]:
t = np.arange(start=0, stop=4*np.pi, step=0.1)
x, y, z = path(t)

In [ ]:
fig = plt.figure()
# Create an Axes3D object, setting the elevation and azimuth
ax = fig.add_subplot(111, projection='3d', elev=20, azim=0)

ax.plot(x, y, z)
plt.title('Desired workspace path')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
plt.show()

Then the corresponding path through the configuration space can be seen as follows. Take note of the drastic changes in the angle configurations. Can you visualize how the robotic arms are moving at the sharp changes in the angle configuration?

In [ ]:
theta1, theta2, q3 = inverse(x, y, z)

In [ ]:
fig = plt.figure()
# Create an Axes3D object, setting the elevation and azimuth
ax = fig.add_subplot(111, projection='3d', elev=20, azim=0)

ax.plot(theta1, theta2, q3)
plt.title('Computed configuration space path')
ax.set_xlabel('$\\theta_1$')
ax.set_ylabel('$\\theta_2$')
ax.set_zlabel('$q_3$')
plt.show()

Run the computed configuration path through the forward kinematics equations as a sanity check.

In [ ]:
cx, cy, cz = forward(theta1, theta2, q3)

In [ ]:
fig = plt.figure()
# Create an Axes3D object, setting the elevation and azimuth
ax = fig.add_subplot(111, projection='3d', elev=20, azim=0)

ax.plot(cx, cy, cz)
plt.title('Computed workspace path')
ax.set_xlabel('$x$')
ax.set_ylabel('$y$')
ax.set_zlabel('$z$')
plt.show()

Again, it's always a good idea to verify that we generated the desired path.

In [ ]:
print(np.allclose(x, cx))
print(np.allclose(y, cy))
print(np.allclose(z, cz))